<a href="https://colab.research.google.com/github/lcpbrandao/Basic_ML_training/blob/master/ML_Bank_Laislla_(Neural_Networks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import os

%env SPARK_HOME=/opt/spark-2.4.3

exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())
sqlContext = SQLContext(sc)

env: SPARK_HOME=/opt/spark-2.4.3
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.5.2 (default, Nov 12 2018 13:43:14)
SparkSession available as 'spark'.


In [ ]:
data = spark.read.csv("bank.csv", header = True, inferSchema = False)

In [ ]:
data.show(2)

+---+------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|   job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|admin.|married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
+---+------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
only showing top 2 rows



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [ ]:
#TRANSFORMAR COLUNA DE job DE CATEGÓRICA PARA ÍNDICE

#(Index - 1a parte)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="job", outputCol="jobIndex")
df = indexer.fit(data).transform(data)
#(OneHotEncoder - 2a parte)
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["jobIndex"],outputCols=["jobVec"])
model = encoder.fit(df)
df2 = model.transform(df)

#TRANSFORMAR COLUNA DE marital DE CATEGÓRICA PARA ÍNDICE
#(Index - 1a parte)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="marital", outputCol="maritalIndex")
df = indexer.fit(df2).transform(df2)
#(OneHotEncoder - 2a parte)
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["maritalIndex"],outputCols=["maritalVec"])
model = encoder.fit(df)
df3 = model.transform(df)

#TRANSFORMAR COLUNA DE marital DE CATEGÓRICA PARA ÍNDICE
#(Index - 1a parte)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="education", outputCol="educationIndex")
df = indexer.fit(df3).transform(df3)
#(OneHotEncoder - 2a parte)
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["educationIndex"],outputCols=["educationVec"])
model = encoder.fit(df)
df4 = model.transform(df)

#TRANSFORMAR COLUNA DE contact DE CATEGÓRICA PARA ÍNDICE
#(Index - 1a parte)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="contact", outputCol="contactIndex")
df = indexer.fit(df4).transform(df4)
#(OneHotEncoder - 2a parte)
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["contactIndex"],outputCols=["contactVec"])
model = encoder.fit(df)
df5 = model.transform(df)

#TRANSFORMAR COLUNA DE poutcome DE CATEGÓRICA PARA ÍNDICE
#(Index - 1a parte)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="poutcome", outputCol="poutcomeIndex")
df = indexer.fit(df5).transform(df5)
#(OneHotEncoder - 2a parte)
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["poutcomeIndex"],outputCols=["poutcomeVec"])
model = encoder.fit(df)
df6 = model.transform(df)

from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="deposit", outputCol="depositIndex")
df = indexer.fit(df6).transform(df6)

df = df.drop('job','marital','education','contact','poutcome',
               'jobIndex','maritalIndex','educationIndex','contactIndex','poutcomeIndex','deposit')

df.show(1)

+---+-------+-------+-------+----+---+-----+--------+--------+-----+--------+--------------+-------------+-------------+-------------+-------------+------------+
|age|default|balance|housing|loan|day|month|duration|campaign|pdays|previous|        jobVec|   maritalVec| educationVec|   contactVec|  poutcomeVec|depositIndex|
+---+-------+-------+-------+----+---+-----+--------+--------+-----+--------+--------------+-------------+-------------+-------------+-------------+------------+
| 59|     no|   2343|    yes|  no|  5|  may|    1042|       1|   -1|       0|(11,[3],[1.0])|(2,[0],[1.0])|(3,[0],[1.0])|(2,[1],[1.0])|(3,[0],[1.0])|         1.0|
+---+-------+-------+-------+----+---+-----+--------+--------+-----+--------+--------------+-------------+-------------+-------------+-------------+------------+
only showing top 1 row



In [ ]:
#SEPARAR BASE DE TREINAMENTO E TESTE

from pyspark.sql import DataFrame
splits = df.randomSplit([0.7, 0.3]) # Divide os dados em dois conjuntos randômicos
train = splits[0] # com 70% dos dados
test = splits[1] # e os 30% restantes

train.show(3)
test.show(3)

+---+-------+-------+-------+----+---+-----+--------+--------+-----+--------+--------------+-------------+------------+-------------+-------------+------------+
|age|default|balance|housing|loan|day|month|duration|campaign|pdays|previous|        jobVec|   maritalVec|educationVec|   contactVec|  poutcomeVec|depositIndex|
+---+-------+-------+-------+----+---+-----+--------+--------+-----+--------+--------------+-------------+------------+-------------+-------------+------------+
| 18|     no|    108|     no|  no| 10|  aug|     167|       1|   -1|       0|(11,[7],[1.0])|(2,[1],[1.0])|   (3,[],[])|(2,[0],[1.0])|(3,[0],[1.0])|         1.0|
| 18|     no|    108|     no|  no|  8|  sep|     169|       1|   -1|       0|(11,[7],[1.0])|(2,[1],[1.0])|   (3,[],[])|(2,[0],[1.0])|(3,[0],[1.0])|         1.0|
| 18|     no|    108|     no|  no|  9|  feb|      92|       1|  183|       1|(11,[7],[1.0])|(2,[1],[1.0])|   (3,[],[])|(2,[0],[1.0])|(3,[2],[1.0])|         1.0|
+---+-------+-------+-------+----+

In [ ]:
# MODELO DE REDES NEURAIS (MULTI LAYER PERCEPTRON)

from pyspark.ml.classification import MultilayerPerceptronClassifier
layers = [10, 20, 2]
# create the trainer and set its parameters
mlp = MultilayerPerceptronClassifier(labelCol="label", featuresCol="scaledFeatures",
maxIter=100, layers=layers, blockSize=100)
mlpModel = mlp.fit ( dfTrain )
result = mlpModel.transform ( dfTest )
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
# Padroniza os dados para ter média igual a zero e desvio-padrão unitário

from pyspark.ml.feature import StandardScaler
scaler = StandardScaler ( inputCol="features", outputCol="scaledFeatures",
withStd=True, withMean=True )
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit ( trainData )
# Normalize each feature to have mean = 0 and unit standard deviation.
scaledTrainData = scalerModel.transform ( trainData )